In [1]:
import pickle
import os
from os import path
from sklearn.linear_model import Perceptron
from sklearn import svm
from numpy.linalg import lstsq
import numpy as np

In [2]:
pcklPath = "/home/iizhaki/oasis/WebStem/pickles/"

mapped = pickle.load(open(pcklPath + "mapped.pck", "rb"))
props = pickle.load(open(pcklPath + "props.pck", "rb"))
hists = pickle.load(open(pcklPath + "hists.pck", "rb"))
refs = pickle.load(open(pcklPath + "refs.pck", "rb"))

In [3]:
tripletsPath = "/home/iizhaki/WebStem/triplets/"
imgsPath = "/home/iizhaki/WebStem/photos/"

groups = [(tripletsPath + f) for f in os.listdir(tripletsPath)]

predXs = [0] * 30
predYs = [0] * 30
predResults = [0] * 30
print groups

['/home/iizhaki/WebStem/triplets/triplets_page_1.log', '/home/iizhaki/WebStem/triplets/triplets_page_2.log']


In [4]:
for group in groups:
    # Blue is lower than Red
    groupN = ""
    groupBR = [set(), set()]
    lower = ""
    upper = ""
    
    with open(group) as f:
        for line in f:               
            splitted = line.split()
            #print splitted
            #break
            
            if len(splitted) < 6:
                continue
            
            if groupN == "":
                groupN = imgsPath + splitted[4].split("/")[2]
                groupY = sorted([f.split("_")[1] for f in os.listdir(groupN) if "org_" in f])
                groupO = sorted([f for f in os.listdir(groupN) if "img_ref_" in f])
                if ("img_ref_" + groupY[0]) in groupO[0]:
                    lower = int(groupO[0].split("_")[3])
                    upper = int(groupO[1].split("_")[3])
                else:
                    lower = int(groupO[1].split("_")[3])
                    upper = int(groupO[0].split("_")[3])
                print "Group:", groupN, groupY
                        
            
            if splitted[5] == "blue":
                if ("ref_" + groupY[0]) in splitted[4]:
                    groupBR[0].add(int(splitted[4].split("_")[3]))
            elif splitted[5] == "red":
                if ("ref_" + groupY[1]) in splitted[4]:
                    groupBR[1].add(int(splitted[4].split("_")[3]))
                    
    print groupBR[0]
    print groupBR[1]
    print lower, upper
    
    
    
    i = 0
    X = []
    y = []

    g0 = int(groupY[0])
    g1 = int(groupY[1])
    lowHist = hists[g0][lower]
    highHist = hists[g1][upper]
    lenHist = len(lowHist)

    # =============================
    if predXs[g0] == 0:
        predXs[g0] = []
        predYs[g0] = []
    
    predXs[g0].append(lowHist);
    predYs[g0].append(1.0);

    predXs[g0].append(highHist);
    predYs[g0].append(0.0);

    for b1 in groupBR[0]:
        predXs[g0].append(hists[g0][b1])
        predYs[g0].append(1.0)    

    for r1 in groupBR[1]:
        predXs[g0].append(hists[g1][r1])
        predYs[g0].append(0.0)


    # =============================
    if predXs[g1] == 0:
        predXs[g1] = []
        predYs[g1] = []
    
    predXs[g1].append(lowHist);
    predYs[g1].append(0.0);

    predXs[g1].append(highHist);
    predYs[g1].append(1.0);

    for b1 in groupBR[0]:
        predXs[g1].append(hists[g0][b1])
        predYs[g1].append(0.0)    

    for r1 in groupBR[1]:
        predXs[g1].append(hists[g1][r1])
        predYs[g1].append(1.0)

    
    

Group: /home/iizhaki/WebStem/photos/group_0 ['1', '2']
set([3105, 3106, 2790, 2993, 3049, 2157, 2926, 3183, 2833])
set([1834, 3021, 2927, 2928, 2929, 3027, 2809, 1722, 2398])
3047 2189
Group: /home/iizhaki/WebStem/photos/group_1 ['1', '2']
set([3107, 2568, 3145, 3051, 3217, 2482, 2627, 2811])
set([1826, 3186, 1832, 2601, 2738, 2894, 3026, 2742])
3048 2181


In [5]:
for i in range(len(predYs)):
    X = predXs[i]
    y = predYs[i]
    
    if X == 0:
        continue
    
    clf = svm.SVC()
    clf.fit(X, y) 
    
    predResults[i] = clf

In [122]:
print X[0], X[1], y[0], y[1]

[0, 6, 0, 0, 0, 588, 60, 306, 4, 0, 242, 0, 17, 1429] [0, 0, 0, 0, 2063, 317, 39, 128, 0, 0, 68, 0, 0, 247] 1.0 0.0


In [123]:
prc = Perceptron()
prc.fit(X, y)

clf = svm.SVC()
clf.fit(X, y) 

thetax, _, _, _ = np.linalg.lstsq(X, y)

In [142]:
allImgs = [f for f in os.listdir(groupN) if "ref_" in f and "img" not in f]

total = 0
right = 0

def predict(data, theta):
    theta = np.array(theta)
    prediction = [np.dot(theta, d)  for d in data]
    return prediction

#ptrain = predict(X_train, thetax)

for b1 in groupBR[0]:
    if int(clf.predict(hists[g0][b1])) == 1:
        right += 1
    total += 1
    
print right * 100.0 / total

# ==============================

total = 0
right = 0

for r1 in groupBR[1]:
    if int(clf.predict(hists[g1][r1])) == 0:
           right += 1
    total += 1

print right * 100.0 / total

# ==============================

total = 0
right = 0
           
for img in allImgs:
    cords = img.split("_")[1: 3]
    if cords[0] == "ref":
        continue
        
    a = int(cords[0])
    b = int(cords[1])
        
    # print cords[0], prc.predict(hists[a][b])[0]
    #if int(cords[0]) ==  int(2 - prc.predict(hists[a][b])[0]):
    res =  np.dot(thetax, np.array(hists[a][b]))
    #if int(cords[0]) ==  int(round(2 - res)):
    if int(cords[0]) ==  int(round(2 - clf.predict(hists[a][b])[0])):
        right += 1
    
    total += 1
    
print right * 100.0 / total

100.0
100.0
81.25
